## Tagging using web-services

In [1]:
import re
import operator
from functools import reduce
import requests
import http.client
import json
from sklearn.feature_extraction.text import CountVectorizer
from enum import Enum
import xml.etree.ElementTree as ET
import pandas as pd
import ast
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [2]:
def read_corpus_file2(filename):
    def prepare_line_record(line):
        items = line.split('\t')
        sentence = items[1]
        label = items[0]
        return sentence, label
    
    
    lines = open(filename, "r",encoding='utf-8').read().splitlines()
    return [prepare_line_record(it) for it in lines]
train_corpus2=read_corpus_file2('text_source/plemo2.0-no-segm.txt')

In [3]:
def wcrf_tagger(text):
    API_ENDPOINT = "http://ws.clarin-pl.eu/nlprest2/base/process"
    body_map = { 
        'lpmn':'any2txt|wcrft2({"guesser":true, "morfeusz2":true})',
        'text': text, 
        'user' : 'moj@adres.mail'
    }
    body_json = json.dumps(body_map)
    r = requests.post(url = API_ENDPOINT, data = body_json) 
    return r.text

In [4]:
def ml_tagger(text):
    API_ENDPOINT = "http://ws.clarin-pl.eu/nlprest2/base/process"
    body_map = { 
        'lpmn':'any2txt|tagger({"lang":"polish"})',
        'text': text, 
        'user' : 'moj@adres.mail'
    }
    body_json = json.dumps(body_map)
    r = requests.post(url = API_ENDPOINT, data = body_json) 
    return r.text

In [5]:
def morphodita_tagger(text):
    API_ENDPOINT = "http://ws.clarin-pl.eu/nlprest2/base/process"
    body_map = { 
        'lpmn':'any2txt|morphoDita({"guesser":false, "allforms":false, "model":"XXI"})',
        'text': text, 
        'user' : 'moj@adres.mail'
    }
    body_json = json.dumps(body_map)
    r = requests.post(url = API_ENDPOINT, data = body_json) 
    r.encoding = "utf-8"
    return r.text

In [6]:
class PartOfSpeech(Enum):
    RZECZOWNIK = 1
    CZASOWNIK = 2
    PRZYMIOTNIK = 3
    LICZEBNIK = 4
    ZAIMEK = 5
    PRZYIMEK = 6
    PRZYSŁÓWEK = 7
    PARTYKUŁA = 8
    WYKRZYKNIK = 9
    SPÓJNIK = 10
    SKRÓT = 11
    OTHER = 12

In [7]:
def get_part_of_speech(tagger_result):
    part_shortcut = tagger_result.split(':')[0]
    
    if part_shortcut in ['subst', 'depr']:
        return PartOfSpeech.RZECZOWNIK
    elif part_shortcut in ['numcol', 'num']:
        return PartOfSpeech.LICZEBNIK
    elif part_shortcut in ['adj', 'adja', 'adjp', 'adjc']:
        return PartOfSpeech.PRZYMIOTNIK
    elif part_shortcut in ['adv', 'qub']:
        return PartOfSpeech.PRZYSŁÓWEK
    elif part_shortcut in ['ppron12', 'ppron3', 'siebie']:
        return PartOfSpeech.ZAIMEK
    elif part_shortcut in ['fin', 'bedzie', 'aglt', 'praet', 'impt', 'imps', 'inf', 'pcon', 'pant', 
                           'ger', 'pact', 'ppas', 'winien']:
        return PartOfSpeech.CZASOWNIK
    elif part_shortcut in ['prep']:
        return PartOfSpeech.PRZYIMEK
    elif part_shortcut in ['conj', 'comp']:
        return PartOfSpeech.SPÓJNIK
    elif part_shortcut in ['brev']:
        return PartOfSpeech.SKRÓT
    elif part_shortcut in ['interjection']:
        return PartOfSpeech.WYKRZYKNIK
    elif part_shortcut in ['burk', 'interp', 'xxx', 'ign','pred']:
        return PartOfSpeech.OTHER
    else:
        print('part_shortcut = ' + part_shortcut + ' NOT FOUND!!!')

In [8]:
def filter_nouns(tagger_output):
    nouns=[]
    for base,form in tagger_output:
        if get_part_of_speech(form)==PartOfSpeech.RZECZOWNIK:
            nouns.append(base)
    return nouns

def filter_adjectives(tagger_output):
    nouns=[]
    for base,form in tagger_output:
        if get_part_of_speech(form)==PartOfSpeech.PRZYMIOTNIK:
            nouns.append(base)
    return nouns

def filter_verbs(tagger_output):
    nouns=[]
    for base,form in tagger_output:
        if get_part_of_speech(form)==PartOfSpeech.CZASOWNIK:
            nouns.append(base)
    return nouns

In [9]:
def process_tagger_output(tagger_result):
    tagging_results=[]
    root = ET.fromstring(tagger_result)
    sentences=root.find('chunk').findall('sentence')
    for sentence in sentences:
        tokens=sentence.findall('tok')
        for token in tokens:
            base=token.find('lex').find('base').text
            grammar_form=token.find('lex').find('ctag').text
            tagging_results.append((base,grammar_form))
    return tagging_results

In [10]:
def persist_tagged_text(dir_name,corpus,tagger,start,to):
    df=pd.DataFrame(zip([],[],[]),columns=['sentence_nr','tagged','label'])
    filename=dir_name+'/'+str(start)+'_'+str(to)+'.csv'
    df.to_csv(filename,index=False)
    sent_index=start
    for text, label in tqdm(corpus[start:to]):
        tagged=str(process_tagger_output(tagger(text)))
        subdf=pd.DataFrame(zip([sent_index],[tagged],[label]),columns=['sentence_nr','tagged','label'])
        sent_index+=1
        subdf.to_csv(filename, mode='a', header=False, index=False)
    return df

In [11]:
def extract_features(dir_name='tagging/wcrf/',source_file_name='merged.csv', x_set_filename='x_features.npy',
                     y_set_filename='y_features.npy', filter_fun=filter_verbs):
    
    LABELS=['meta_minus_m','meta_minus_s','meta_plus_m','meta_plus_s','meta_amb','meta_zero']
    
    def corpus_from_file():
         return pd.read_csv(dir_name+source_file_name) 
    
    def create_corpus_list():
        corpus_list=[]
        read_from_files=corpus_from_file() 
        tagged_column=read_from_files['tagged']
        for row in tagged_column:
            row_list=ast.literal_eval(row)
            corpus_list.extend(filter_fun(row_list))
        return corpus_list
    
    def build_bags_of_words_vectorizer(corpus):
        vectorizer = CountVectorizer()
        vectorizer.fit_transform(corpus)
        print(vectorizer.get_feature_names())
        return vectorizer

    def translate_to_bags_of_words(text,vectorizer):
        return vectorizer.transform([text]).toarray()
    
    def save_y_features():
        labels=read_from_files=corpus_from_file()['label'].tolist()
        y_np_array=np.ones(shape=(1,1))
        for label in labels:
            y_np_array=np.append(y_np_array,[[LABELS.index(label)]], axis=0)
        np.save(dir_name+y_set_filename,y_np_array[1:])
        
    def save_x_features():
        corp=create_corpus_list()
        vectorizer=build_bags_of_words_vectorizer(corp)
        x_np_array=translate_to_bags_of_words('',vectorizer)
        read_from_files=corpus_from_file() 
        tagged_column=read_from_files['tagged']
        for row in tagged_column:
            row_list=ast.literal_eval(row)
            filtered=str(filter_fun(row_list))
            translated=translate_to_bags_of_words(filtered,vectorizer)
            x_np_array=np.append(x_np_array,translated, axis=0)
        np.save(dir_name+x_set_filename,x_np_array[1:])
        
    save_y_features()
    save_x_features()

In [12]:
def merge_files(dir_name,output,filenames):
    base=pd.DataFrame(zip([],[],[]),columns=['sentence_nr','tagged','label'])
    for file in filenames:
        read_df=pd.read_csv(dir_name+file+'.csv')
        base=base.append(read_df)
    base.to_csv(dir_name+output+'.csv',index=False)

In [13]:
def prepare_merging_list():
    step=100
    start=3000
    end=9100
    mergelist=[]
    while start< end:
        mergelist.append(str(start)+"_"+str(start+step))
        start+=step
    return mergelist
mergelist=prepare_merging_list()

## EXECUTION

In [15]:
krnnt=pd.read_csv('tagging/krnnt/krnn.csv')
krnnt

,sentence_nr,tagged,label
0,0,"[('wykład', 'subst:pl:nom:m3'), ('strasznie', ...",meta_minus_m
1,1,"[('ogólnie', 'adv:pos'), ('bardzo', 'adv:pos')...",meta_plus_s
2,2,"[('pan', 'subst:sg:nom:m1'), ('Krzysztof', 'su...",meta_plus_m
3,3,"[('Wiesław', 'subst:sg:nom:m1'), ('Dudek', 'su...",meta_amb
4,4,"[('Jankowska', 'subst:sg:nom:f'), ('Elżbieta',...",meta_minus_s
...,...,...,...
9319,9319,"[('bardzo', 'adv:pos'), ('dobry', 'adj:sg:nom:...",meta_plus_m
9320,9320,"[('nasz', 'adj:sg:acc:m3:pos'), ('pobyt', 'sub...",meta_plus_m
9321,9321,"[('być', 'praet:sg:m1:imperf'), ('być', 'aglt:...",meta_minus_m
9322,9322,"[('minus', 'subst:pl:nom:m3'), ('(', 'interp')...",meta_minus_m


In [16]:
# persisted=persist_tagged_text('tagging/wcrf2',train_corpus2,wcrf_tagger,0,9100)

In [17]:
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,2836,3000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,3000,3500)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,3500,4000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,4000,4500)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,4500,5000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,5000,6000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,6000,7000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,7000,8000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,8000,9000)
# persist_tagged_text('tagging/wcrf3',train_corpus2,morphodita_tagger,9000,9100)

In [23]:
mg=['0_3000','3000_9300']

In [24]:
merge_files('tagging/wcrf2guesser/','merged',mg)

In [20]:
# v3=pd.read_csv('tagging/wcrf2guesser/v3.csv',sep=';')

In [21]:
# base=pd.DataFrame(zip([],[],[]),columns=['sentence_nr','tagged','label'])
# read_df=pd.read_csv('tagging/wcrf2guesser/0_3000.csv')
# base=base.append(read_df)
# base=base.append(v3)
# base.to_csv('tagging/wcrf2guesser/merged.csv',index=False)

In [26]:
extract_features(dir_name='tagging/wcrf2guesser/',source_file_name='merged.csv', x_set_filename='x_verbs.npy',
                     y_set_filename='y_verbs.npy', filter_fun=filter_verbs)
extract_features(dir_name='tagging/wcrf2guesser/',source_file_name='merged.csv', x_set_filename='x_nouns.npy',
                     y_set_filename='y_nouns.npy', filter_fun=filter_nouns)
extract_features(dir_name='tagging/wcrf2guesser/',source_file_name='merged.csv', x_set_filename='x_adj.npy',
                     y_set_filename='y_adj.npy', filter_fun=filter_adjectives)

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

...

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F

part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT FOUND!!!
part_shortcut = interj NOT F